# Debugging LangGraph Interrupt Events

This notebook is designed to investigate the structure of the `event_data` object produced by the LangGraph workflow, specifically the `__interrupt__` event. 

The goal is to find the correct attribute to use for a unique ID within the `Interrupt` object, to fix the `AttributeError: 'Interrupt' object has no attribute 'ns'` that occurs in `src/server/app.py`.

In [1]:
import asyncio
import nest_asyncio
nest_asyncio.apply()
import os
from langchain_core.messages import HumanMessage

# Set environment variables to match the deployed environment
os.environ['LANGGRAPH_CHECKPOINT_SAVER'] = 'false'
os.environ['BASIC_MODEL__PLATFORM'] = 'google_aistudio'
os.environ['BASIC_MODEL__MODEL'] = 'gemini-2.5-flash'
os.environ['BASIC_MODEL__API_KEY'] = os.getenv('GEMINI_API_KEY', '') # Assumes GEMINI_API_KEY is in the environment

from src.graph.builder import build_graph_with_memory

async def main():
    print("Building graph...")
    graph = build_graph_with_memory()
    
    # Define the initial input for the workflow
    workflow_input = {
        "messages": [HumanMessage(content="give me a detailed overview and analysis of Agentech, a tech company in Oklahoma City")],
        "auto_accepted_plan": False, # Ensure the graph will interrupt for feedback
        "enable_background_investigation": True,
    }
    
    # Define a config with a thread_id
    config = {"configurable": {"thread_id": "debug-thread-123"}} 
    
    print("Streaming workflow...")
    try:
        async for event in graph.astream(workflow_input, config=config, stream_mode=["updates"]):
            print("--- NEW EVENT ---")
            print(event)
            
            # Check if this event is the interrupt we are looking for
            if isinstance(event, dict) and "__interrupt__" in event:
                print("\n***********************************")
                print("*** INTERRUPT EVENT FOUND   ***")
                print("***********************************\n")
                
                interrupt_object = event["__interrupt__"][0] # The Interrupt object is inside a list
                
                print(f"Interrupt Object: {interrupt_object}")
                print(f"Interrupt Object Type: {type(interrupt_object)}\n")
                
                # Use dir() to inspect all available attributes and methods
                print(f"All attributes of the Interrupt object:")
                print(dir(interrupt_object))
                print("\n***********************************")
                print("*** END OF INTERRUPT ANALYSIS ***")
                print("***********************************\n")
                # We found what we need, so we can stop.
                break
    except Exception as e:
        print(f"An error occurred during the stream: {e}")

print("Running notebook code...")
asyncio.run(main())
print("Notebook execution finished.")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\pault\anaconda3\envs\app_project\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\pault\anaconda3\envs\app_project\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\pault\anaconda3\envs\app_project\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.sta

Running notebook code...
Building graph...
Streaming workflow...
--- NEW EVENT ---
('updates', {'coordinator': {'messages': [HumanMessage(content='give me a detailed overview and analysis of Agentech, a tech company in Oklahoma City', additional_kwargs={}, response_metadata={}, id='03a6cb18-dcb5-4417-8de2-e79cfbd32de1')], 'locale': 'en-US', 'research_topic': 'detailed overview and analysis of Agentech, a tech company in Oklahoma City', 'resources': []}})


Key 'examples' is not supported in schema, ignoring


--- NEW EVENT ---
('updates', {'background_investigator': {'background_investigation_results': '## Agentech - SPEEDA Edge\n\nIn January 2025, the company was selected to join NVIDIA\'s Inception Program, through which it will gain access to NVIDIA\'s developer courses, cloud computing credits, software and hardware discounts, and engineering resources. These tools will help Agentech enhance its AI agents that automate insurance claims.\n\nSectors:\n\nFinancial Services\n\nThemes:\n\nB2B SaaS\n\nGenerative AI (GenAI)\n\nAI/ML\n\nHQ location:\n\nOklahoma City OK USA\n\nFounded year:\n\n2023\n\nEmployees:\n\n11-50\n\nWebsite:\n\nIPO status: [...] Agentech is an InsurTech company specializing in AI-powered workforce solutions for insurance claims processing. Its platform integrates seamlessly with existing claims management systems, automating key tasks such as document review, compliance checks, and data extraction. At the core of its offering is an ensemble of multimodal large language m